In [3]:
!pip install --upgrade numpy scipy scikit-learn
!sudo apt install libopenblas-dev  # Install BLAS backend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
Reading package lists..

In [4]:
# %% [markdown]
# ## ML Energy Efficiency Analysis in Colab

# %% [code]
# Install required packages
!pip install psutil  # Needed for system monitoring

# %% [code]
import numpy as np
import time
import psutil
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
from IPython.display import display, Markdown

# %% [code]
class EnergyEfficiencyAnalyzer:
    def __init__(self, output_dir):
        self.results = {
            'architectures': [],
            'training_time': [],
            'energy_consumption': [],
            'accuracy': []
        }
        self.output_dir = output_dir

    def generate_dataset(self, n_samples=1000, n_features=10):
        np.random.seed(42)
        X = np.random.randn(n_samples, n_features)
        y = (X[:, 0] + X[:, 1] > 0).astype(int)
        return X, y

    def measure_ml_performance(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        start_cpu = psutil.cpu_percent()
        start_memory = psutil.virtual_memory().used
        start_time = time.time()

        model = MLPClassifier(
            hidden_layer_sizes=(64, 32),
            max_iter=100,
            random_state=42
        )
        model.fit(X_train_scaled, y_train)

        end_time = time.time()
        end_cpu = psutil.cpu_percent()
        end_memory = psutil.virtual_memory().used

        training_time = end_time - start_time
        cpu_usage = end_cpu - start_cpu
        memory_usage = end_memory - start_memory

        accuracy = model.score(X_test_scaled, y_test)
        energy_proxy = (cpu_usage * memory_usage) / 1e9

        self.results['architectures'].append('MLP Classifier')
        self.results['training_time'].append(training_time)
        self.results['energy_consumption'].append(energy_proxy)
        self.results['accuracy'].append(accuracy)

        return {
            'training_time': training_time,
            'energy_proxy': energy_proxy,
            'accuracy': accuracy
        }

    def visualize_results(self):
        plt.figure(figsize=(15, 5))

        plt.subplot(131)
        plt.scatter(self.results['training_time'], self.results['energy_consumption'])
        plt.title('Training Time vs Energy')

        plt.subplot(132)
        plt.bar(self.results['architectures'], self.results['accuracy'])
        plt.title('Model Accuracy')

        plt.subplot(133)
        efficiency_ratio = [acc/energy if energy > 0 else 0
                          for acc, energy in zip(self.results['accuracy'],
                                               self.results['energy_consumption'])]
        plt.bar(self.results['architectures'], efficiency_ratio)
        plt.title('Accuracy/Energy Efficiency')

        plt.tight_layout()
        plot_path = os.path.join(self.output_dir, 'ml_energy_efficiency.png')
        plt.savefig(plot_path)
        plt.close()

        # Display in notebook
        display(Markdown(f"**Saved plot to:** `{plot_path}`"))
        display(plt.imread(plot_path))

    def generate_report(self):
        report_data = {
            'Total Runs': len(self.results['architectures']),
            'Average Training Time': np.mean(self.results['training_time']),
            'Average Energy Consumption': np.mean(self.results['energy_consumption']),
            'Average Accuracy': np.mean(self.results['accuracy'])
        }

        report_path = os.path.join(self.output_dir, 'performance_report.txt')
        with open(report_path, 'w') as f:
            f.write("Machine Learning Energy Efficiency Report\n")
            f.write("=========================================\n\n")
            for key, value in report_data.items():
                f.write(f"{key}: {value}\n")

        # Display report contents
        display(Markdown(f"**Report generated at:** `{report_path}`"))
        with open(report_path, 'r') as f:
            display(Markdown(f"```\n{f.read()}\n```"))

        return report_data

# %% [code]
# Main execution
output_dir = "/content/energy_efficiency_analysis"
os.makedirs(output_dir, exist_ok=True)

analyzer = EnergyEfficiencyAnalyzer(output_dir)
X, y = analyzer.generate_dataset(n_samples=5000, n_features=10)

# Run analysis
print("🚀 Running ML Performance Analysis...")
results = analyzer.measure_ml_performance(X, y)

# Display immediate results
display(Markdown("### Immediate Results:"))
for metric, value in results.items():
    display(Markdown(f"- **{metric.replace('_', ' ').title()}:** {value:.4f}"))

# Generate visualizations and report
analyzer.visualize_results()
final_report = analyzer.generate_report()

# Final summary
display(Markdown("### Final Summary:"))
for metric, value in final_report.items():
    display(Markdown(f"- **{metric.replace('_', ' ').title()}:** {value:.4f}"))

# %% [markdown]
# ### Accessing Results
# - Plots saved to: `/content/energy_efficiency_analysis/ml_energy_efficiency.png`
# - Report saved to: `/content/energy_efficiency_analysis/performance_report.txt`
#
# To download results:
# ```python
# from google.colab import files
# !zip -r results.zip {output_dir}
# files.download('results.zip')
# ```


🚀 Running ML Performance Analysis...


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


### Immediate Results:

- **Training Time:** 8.5737

- **Energy Proxy:** 3.5439

...

### Final Summary:

- **Total Runs:** 1.0000

- **Average Training Time:** 8.5737

- **Average Energy Consumption:** 3.5439

- **Average Accuracy:** 0.9910

In [8]:
import baler
print(dir(baler))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'baler', 'modules', 'perform_compression', 'perform_decompression', 'perform_diagnostics', 'perform_plotting', 'perform_training', 'print_info']


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Create directory if missing
!mkdir -p /train_data

# 1. Upload your dataset here using Colab's file explorer
#    (left sidebar ➔ 📁 folder icon)

# 2. Verify dataset structure
assert os.path.exists('/train_data'), "Dataset not found!"

# 3. Create generator
calibration_datagen = ImageDataGenerator(rescale=1./255)

calibration_generator = calibration_datagen.flow_from_directory(
    '/content/train_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

print(f"✅ Found {calibration_generator.samples} calibration images")

Found 4 images belonging to 3 classes.
✅ Found 4 calibration images


In [18]:
# First install baler if needed (uncomment if required)
# !pip install git+https://github.com/yourusername/baler.git

from baler import perform_compression
import tensorflow as tf
import numpy as np

# Load model with proper Colab path
try:
    model = tf.keras.models.load_model("/content/cat_dog_model.h5")
except Exception as e:
    raise FileNotFoundError(f"Model not found at /content/cat_dog_model.h5 - Upload model first!\n{str(e)}")

# Compression config with validation
config = {
    'quantization': {
        'bits': 8,  # Supported: 8, 16
        'per_channel': True
    },
    'pruning': {
        'sparsity': 0.5,  # Between 0-1
        'block_size': (1, 1)
    },
    'format': 'h5'
}

# Generate calibration data from your prepared generator
# Using 10 batches (320 samples) assuming batch_size=32
calibration_samples = []
for _ in range(10):
    batch, _ = next(calibration_generator)
    calibration_samples.append(batch)
calibration_data = np.concatenate(calibration_samples)

# Perform compression with error handling
try:
    compressed_model = perform_compression(
        model=model,
        config=config,
        calibration_data=calibration_data,  # Now properly defined
        verbose=True
    )

    # Save with proper permissions
    compressed_model.save("/content/optimized_cat_dog_model.h5")
    print("✅ Compression successful! Saved to /content/optimized_cat_dog_model.h5")

except Exception as e:
    print(f"❌ Compression failed: {str(e)}")
    print("Common fixes:")
    print("1. Check baler installation")
    print("2. Verify calibration data shape matches model input")
    print("3. Ensure TensorFlow version compatibility")

❌ Compression failed: perform_compression() got an unexpected keyword argument 'model'
Common fixes:
1. Check baler installation
2. Verify calibration data shape matches model input
3. Ensure TensorFlow version compatibility


In [ ]:
import os
import time

# Get file sizes
original_size = os.path.getsize("/content/cat_dog_model.h5") / 1024  # KB
optimized_size = os.path.getsize("/content/cat_dog_model_optimized.h5") / 1024  # KB

# Load optimized model
opt_model = keras.models.load_model("/content/cat_dog_model_optimized.h5")

# Measure inference time
def measure_time(model, sample):
    start = time.time()
    model.predict(sample)
    return time.time() - start

sample = np.expand_dims(x_test[0], axis=0)

original_time = measure_time(model, sample)
optimized_time = measure_time(opt_model, sample)

# Print comparison
print(f"🔹 Original Model Size: {original_size:.2f} KB")
print(f"🔹 Optimized Model Size: {optimized_size:.2f} KB")
print(f"⚡ Inference Time (Original): {original_time:.5f} sec")
print(f"⚡ Inference Time (Optimized): {optimized_time:.5f} sec")


In [ ]:
import matplotlib.pyplot as plt

labels = ["Original", "Optimized"]
sizes = [original_size, optimized_size]
times = [original_time, optimized_time]

plt.figure(figsize=(10, 4))

# Storage Reduction
plt.subplot(1, 2, 1)
plt.bar(labels, sizes, color=["red", "green"])
plt.ylabel("Model Size (KB)")
plt.title("Model Storage Optimization")

# Inference Time Reduction
plt.subplot(1, 2, 2)
plt.bar(labels, times, color=["blue", "orange"])
plt.ylabel("Inference Time (sec)")
plt.title("Inference Speed Optimization")

plt.show()
